<a href="https://colab.research.google.com/github/BrotherKim/KAIST_CODE/blob/master/SEP531/term_5/generate_small_testset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#변수 정의

In [1]:
REGION_LIST = ['ks', 'jr', 'kw', 'cc'] # jj는 특이케이스라 따로 처리
REGION_LIST = ['cc'] # jj는 특이케이스라 따로 처리

ROOT_DIR = '/content/drive/MyDrive/KAIST/SEP531/kor_%s_labeled'
TRAIN_DIR = '%s/train' % ROOT_DIR
VALID_DIR = '%s/valid' % ROOT_DIR

NUM_TRAIN = 171
NUM_VALID = 58

SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_finetuned_data_std.pt'

#실행 환경#

- Python >= 3.6
- PyTorch >= 1.70
- Transformers = 3.0.2
- Colab
- batch size = 64 (convertable)
- epochs = 10 (convertable)

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 1.4 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 6.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595732 sha256=141fbb87106b8760939ec37f01a6d6177427d4f3102161005a6f5af5056b85ca
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 8.4 MB/s 
     |████████████████████████████████| 769 kB 6.9 MB/s 
     |████████████████████████████████| 895 kB 57.4 MB/s 
     |████████████████████████████████| 3.0 MB 58.6 MB/s 


#KoBERT 다운로드#

In [3]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-i746rl3p
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-i746rl3p
     |████████████████████████████████| 4.9 MB 6.5 MB/s 
     |████████████████████████████████| 3.4 MB 51.9 MB/s 
     |████████████████████████████████| 61 kB 653 kB/s 
     |████████████████████████████████| 596 kB 47.9 MB/s 
     |████████████████████████████████| 3.3 MB 49.9 MB/s 
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13136 sha256=3cdc5ca91a215b7991c190ca014b409db82672b39974e81503e2fd11ea216a04
  Stored in directory: /tmp/pip-ephem-wheel-cache-5vicg_4m/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: tokenizers
    Found existi

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
#GPU 사용
device = torch.device("cuda:0")

In [7]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


#데이터셋 전처리#

In [8]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# 터미널 커맨드를 문자열로 반환해주는 함수 작성

import subprocess

def GetShellCmdStdOut(command):
  cmd = ['sh', '-c', command]
  fd_popen = subprocess.Popen(cmd, stdout=subprocess.PIPE).stdout 
  data = fd_popen.read().strip() 
  fd_popen.close()

  retval = data.decode('utf-8') 
  return retval

In [10]:
import pandas as pd

def GenDataset(DIR, json_list):
  retval = []

  for fn in json_list:
    #print('%s\n' % (fn))
    fd = pd.read_csv('%s/%s' % (DIR, fn), sep='\t', header=None)
    # [0]데이터가 표준어, [1]데이터가 사투리, [2]가 감정
    for q, label in zip(fd[0], fd[2])  :
      #print('%s, %s\n' % (q, label))
      data = []
      data.append(q)
      data.append(str(label))
      retval.append(data)

  return retval

In [11]:
dataset_train = []

for region in REGION_LIST:
  train_dir = TRAIN_DIR % (region)
  print(train_dir)
  %pushd $train_dir
  tl = GetShellCmdStdOut('find | grep answer').split('\n')
  train_json_list = [x.replace('./', '') for x in tl]
  dataset_add = GenDataset(train_dir, train_json_list)
  #dataset_train = dataset_train + dataset_add[:11250]
  dataset_train = dataset_train + dataset_add[45000:]
  %popd

dataset_valid = []

for region in REGION_LIST:
  valid_dir = VALID_DIR % (region)
  print(valid_dir)
  %pushd $valid_dir
  vl = GetShellCmdStdOut('find | grep answer').split('\n')
  valid_json_list = [x.replace('./', '') for x in vl]
  dataset_add = GenDataset(valid_dir, valid_json_list)
  #dataset_valid = dataset_valid + dataset_add[:3750]
  dataset_valid = dataset_valid + dataset_add[15000:]
  %popd

/content/drive/MyDrive/KAIST/SEP531/kor_cc_labeled/train
/content/drive/MyDrive/KAIST/SEP531/kor_cc_labeled/train
/content
popd -> /content
/content/drive/MyDrive/KAIST/SEP531/kor_cc_labeled/valid
/content/drive/My Drive/KAIST/SEP531/kor_cc_labeled/valid
/content
popd -> /content


In [12]:
print(len(dataset_train))
print(len(dataset_valid))

10154
4954


In [13]:
dataset_train[:10]

[['음~ 너는 연예인 중에 이상형이 있다면 누구야?', '4'],
 ['물론 연예인 분들이 많아서 이쁘신 분들은 많지만 지금', '3'],
 ['이 순간 딱 떠오른 이상형은 박보영인 거 같애 너도 연예인 중에 이상형이 있어?', '1'],
 ['나는 그때 뷰티 인사이드 나왔을 때 이민기가 너무 좋아서 그런', '5'],
 ['그~ 역할도 외모도 키도 다 내 이상형이야.', '5'],
 ['너는 왜 박보영이야?', '1'],
 ['어 평소에 내 이상형이', '1'],
 ['아담하고 귀여운 사람을 좋아하는 스타일이여가지고 박보영을 선택하게 된 거 같애', '4'],
 ['너는 결혼에 대해서는 어떻게 생각해?', '1'],
 ['결혼을 하고 싶냐고?', '1']]

In [14]:
%cd /content/drive/MyDrive/KAIST/SEP531

/content/drive/MyDrive/KAIST/SEP531


In [15]:
cnt = [0, 0, 0, 0, 0, 0, 0, 0]
raw = [[], [], [], [], [], [], [], []]
for d in dataset_train:
  i = int(d[1])
  cnt[i] = cnt[i] + 1
  raw[i].append(d[0])
for d in dataset_valid:
  i = int(d[1])
  cnt[i] = cnt[i] + 1
  raw[i].append(d[0])

In [16]:
print(len(raw[0]))
print(len(raw[1]))
print(len(raw[2]))
print(len(raw[3]))
print(len(raw[4]))
print(len(raw[5]))
print(len(raw[6]))

1580
2115
510
2579
5455
1733
1136


In [32]:
dataset = []

for i in range(0, 954):
  d = []
  d.append(raw[0][i])
  d.append(0)
  dataset.append(d)

for i in range(0, 1555):
  d = []
  d.append(raw[1][i])
  d.append(1)
  dataset.append(d)

for i in range(0, 358):
  d = []
  d.append(raw[2][i])
  d.append(2)
  dataset.append(d)

for i in range(0, 1516):
  d = []
  d.append(raw[3][i])
  d.append(3)
  dataset.append(d)

for i in range(0, 3849):
  d = []
  d.append(raw[4][i])
  d.append(4)
  dataset.append(d)

for i in range(0, 938):
  d = []
  d.append(raw[5][i])
  d.append(5)
  dataset.append(d)

for i in range(0, 830):
  d = []
  d.append(raw[6][i])
  d.append(6)
  dataset.append(d)  

In [33]:
print(len(dataset))

10000


In [34]:
cnt

[1580, 2115, 510, 2579, 5455, 1733, 1136, 0]

In [38]:
!rm std_test_small.csv

In [41]:
!ls

 연속대화_10000.csv
 한국어_연속적_대화_데이터셋_변환1.xlsx
 한국어_단발성_대화_데이터셋_변환1.xlsx
 4차년도.csv
 4_utf8.csv
 4차년도_utf8.csv
 5차년도_2차.csv
 5_2_utf8.csv
 5차년도_2차_utf8.csv
 5차년도.csv
 5_utf8.csv
 5차년도_utf8.csv
 cc_10000.csv
 한국어_연속적_대화_데이터셋.csv
 단발성대화.csv
 HAK_TO_HYUNG
 hw3_cp
 hw4_cp
 jj_10000.csv
 jr_10000.csv
 KoBERT_emotion_finetuned_data_added.pt
'KoBERT_emotion_finetuned_data_added.pt의 사본'
 KoBERT_emotion_finetuned_data_jj.pt
 KoBERT_emotion_finetuned_data_jr.pt
 KoBERT_emotion_finetuned_data_ks.pt
 KoBERT_emotion_finetuned_data_std.pt
 KoBERT_emotion_finetuned.pt
 kor_cc_labeled
 kor_jj_labeled
 kor_jr_labeled
 kor_ks_labeled
 kor_kw_labeled
 ks_10000.csv
 kw_10000.csv
 model
 감성대화말뭉치_output.csv
 Progress-Report_20213730김학성_20213731김형.pptx
 std_test.csv
 std_test_small.csv
 감성대화말뭉치_train_변환1.xlsx
 감성대화말뭉치_train.csv
 감성대화말뭉치_train.xlsx
 한ᄀ

In [40]:
with open('std_test_small.csv', 'w') as f:
  for d in enumerate(dataset):
    f.write('%s|%s\n' % (d[0], d[1]))

  

In [37]:
for l, d in enumerate(dataset):
  print('%s|%s\n' % (d[0], d[1]))    

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
칼국수랑 거기는 버섯으로 육수를 내|4

칼국수랑 보쌈인데 속 나도 &company-name1& 되게 좋아하는데 &company-name1&랑 되게 다른 맛이야.|4

되게 다르게 맛있어서 비교를 할 수가 없는 게|4

그래 주소 찍어 줘 그러면은 음식중에서 한식 양식 일식 중식 뭐 좋아해?|4

난 다 좋아해.|4

그래도 그 중에 하나 있어?|4

그래도 한국인이다 보니깐 한식 좋아하기는 하는데 두 번째론 일식 좋아하지 나도 초밥 좋아하는데 여러 군데 있거든|4

요즘은 또 방어 철이다 보니까는 조만간 또 방어 먹겠네.|4

우리 작년 이브에 먹은 회가 뭐였지 광어 우럭 먹었나?|4

우리 그때 광어 &company-name7& 가서 어 광어 우리 같이 먹었었던 거 같앴었는데|4

방어를 한 번도 안 먹어 봤어?|4

어|4

오빠는 오빠는 근데 술을 안 좋아해서 다른가?|4

그 소주는 잘 안 먹지만 소맥은 그래도 어느 정도 먹으면 좋긴 하더라.|4

나는 근데 매운탕 때문에 술 먹는 것 같애 왜냐하면 술 먹고 매운탕 먹으면 뭔가 기분이 좋은 것 같아서|4

나는 먹을 걸 되게 좋아하는데 대전에는 막 맛집이 많진 않은거 같애 다 인스타 맛집 사진용 맛집 많은 거 같고|4

&name3&는 매운 거 잘 먹잖아.|4

갑자기 막 뭐라 그러더라 &company-name5& 어 이런 데 닭발은 솔직히 너무 맵잖아.|4

한신 닭발도 백종원 거라 그러더라고 근데 한신 닭발 너무 달아서 난 별로 맛이 없고|4

아직도 그렇게 코로나 심해도 잘 되는 집은 잘되고 맛집은 역시 맛집이구나 싶더라고|4

{clearing} 요즘 뭐 에스엔에스로 이렇게 좀 떠있는 그런 맛집들 있잖아.|4

오히려 이런 홍보 안 하는 &company-name2& 그냥 &company-name1& 같은 데가 진짜 좀 진정한 맛집인 것 같애|4

나 한 몇 군데 있는 것 같애 한신도 가봤고 그다음에 새마을 그~ 고기집 뭐더라 기억이